In [2]:
from pulp import *
import pandas as pd
from constraint import *

### Question 2

In [6]:
#Create a list of all the products
Products = ["A","B"]

#Create a dictionary of the minimum quantity for products
MinQ = {"A" : 75, "B" : 0}

#Create a dictionary of the profits per unit for products
Profits = {"A" : 470, "B" : 420}

#Create a list of all the machines
Machines = ["M1","M2","M3"]

#Create a Dictionary of Availability
#Same value for each machine!
MachinesAvailability = 336

#Create a list for amount used of each raw material for each product
CapacityUsed = {"A": {"M1": 2, "M2": 0, "M3": 2},
              "B":{"M1": 0, "M2": 2.5, "M3": 1.5}
             }

# Create the 'prob' variable to contain the problem data
prob = LpProblem("2 products 3 machines", LpMaximize)

# Create the Variables
product_vars = {p:LpVariable("Prod_{}".format(p),lowBound=MinQ[p],upBound=140,cat=LpContinuous) for p in Products}

# The objective function is added to 'prob' first
prob += lpSum([Profits[i]*product_vars[i] for i in Products]) - 50000, "Total Revenue of Production Plan"

# We can enter the constraints that relate to limited amount of material
for r in Machines:
    prob += lpSum([product_vars[i]*CapacityUsed[i][r] for i in Products]) <= MachinesAvailability, r

# The problem data is written to an .lp file
prob.writeLP("MachinesProducts.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print("Total Profit of Plan = ", value(prob.objective))

for constraint in prob.constraints:
        print(prob.constraints[constraint].name, prob.constraints[constraint].value())
        print(prob.constraints[constraint].name, prob.constraints[constraint].constant)

Status: Optimal
Prod_A = 75.0
Prod_B = 124.0
Total Profit of Plan =  37330.0
M1 -186.0
M1 -336
M2 -26.0
M2 -336.0
M3 0.0
M3 -336.0


In [3]:
prob

2 products 3 machines:
MAXIMIZE
470*Prod_A + 420*Prod_B + 0
SUBJECT TO
M1: 2 Prod_A <= 336

M2: 2.5 Prod_B <= 336

M3: 2 Prod_A + 1.5 Prod_B <= 336

VARIABLES
75 <= Prod_A <= 140 Continuous
Prod_B <= 140 Continuous

### Question 3

In [4]:
#start by parsing the data from excel
parameters_df = pd.read_excel('HW1 Modeling Problems- Student.xlsx', sheet_name='Diet', skiprows=2, usecols='B:G', index_col=0)
parameters_dict = parameters_df.to_dict()

#Create a list of all the products
Products = list(parameters_df.index)

#Create a dictionary of the price per unit for products
Prices = parameters_dict.pop('Unit Cost')

#Create a list of all the vitamins
Vitamins = list(parameters_dict.keys())
Vitamins.pop(0)

#Create a Dictionary of Availability
#Same value for each machine!
VitaminsNeed = 1

#Create a list for amount used of each raw material for each product
VitaminsContent = parameters_dict

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Vitamins", LpMinimize)

# Create the Variables
product_vars = LpVariable.dicts("Prods",Products,lowBound=0,upBound=None,cat=LpContinuous)

# The objective function is added to 'prob' first
prob += lpSum([Prices[i]*product_vars[i] for i in Products]), "Total Cost of Diet Plan"

# We can enter the constraints that relate to getting the required amounts of vitamins
for r in Vitamins:
    prob += lpSum([product_vars[i]*VitaminsContent[r][i] for i in Products]) >= VitaminsNeed, r

# The problem data is written to an .lp file
prob.writeLP("Vitamins.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Total Cost of Diet = ", value(prob.objective))

for constraint in prob.constraints:
        print(prob.constraints[constraint].name, prob.constraints[constraint].value())
        print(prob.constraints[constraint].name, prob.constraints[constraint].constant)

Status: Optimal
Prods_Beef = 0.0
Prods_Chicken = 2.5
Prods_Fish = 0.0
Prods_Ham = 0.0
Prods_Meat_Loaf = 3.3333333
Prods_Turkey = 0.0
Total Cost of Diet =  14.374999925000001
C -1.0000000050247593e-08
C -1.0
B1 -5.000000025123796e-09
B1 -1.0
B2 -5.000000025123796e-09
B2 -1.0


In [5]:
prob

Vitamins:
MINIMIZE
4.16*Prods_Beef + 2.75*Prods_Chicken + 3.28*Prods_Fish + 2.91*Prods_Ham + 2.25*Prods_Meat_Loaf + 2.45*Prods_Turkey + 0.0
SUBJECT TO
C: 0.2 Prods_Beef + 0.1 Prods_Fish + 0.4 Prods_Ham + 0.3 Prods_Meat_Loaf
 + 0.2 Prods_Turkey >= 1

B1: 0.1 Prods_Beef + 0.2 Prods_Chicken + 0.15 Prods_Fish + 0.35 Prods_Ham
 + 0.15 Prods_Meat_Loaf + 0.15 Prods_Turkey >= 1

B2: 0.15 Prods_Beef + 0.2 Prods_Chicken + 0.1 Prods_Fish + 0.1 Prods_Ham
 + 0.15 Prods_Meat_Loaf + 0.1 Prods_Turkey >= 1

VARIABLES
Prods_Beef Continuous
Prods_Chicken Continuous
Prods_Fish Continuous
Prods_Ham Continuous
Prods_Meat_Loaf Continuous
Prods_Turkey Continuous

### Question 4

In [42]:
#Create a list of all the fuel mixes
Fuels = ["F1","F2", "F3"]

#Create a list of all the raw materials
RawGas = ["R1", "R2", "R3", "R4"]

#Create a dictionary of the minimum quantity for fuel mixes
MinQ = {"F1" : 0, "F2" : 0, "F3" : 15000}

#Create a dictionary of the maximum quantity for fuel mixes
MaxQ = {"F1" : 10000, "F2" : 999999, "F3" : 999999}

#Create a dictionary of the unit price of the fuel mixes
Prices = {"F1" : 45.15, "F2" : 42.95, "F3" : 40.99}

#Create a list of all the quantities of raw gas types in fuel mixes
Quantities = ["R1_1","R2_1","R3_1","R4_1",
              "R1_2","R2_2","R3_2","R4_2",
              "R1_3","R2_3","R3_3","R4_3"]

#Create a list for amount used of each raw material for each product
OctaneConstraints = {"F1": {"R1":-27, "R2":-9,"R3":-4,"R4":4},
                     "F2": {"R1":-22, "R2":-4, "R3":1, "R4":9},
                     "F3": {"R1":-17, "R2":1, "R3":6, "R4":14}}

#Create a dictionary of the capacity constraints
Capacity = {"R1":4000, "R2":5050, "R3":7100, "R4":4300}
                     
#Compute total cost of raw material (raw gas types)
Unit_Cost = [31.02, 33.15, 36.35, 38.7]
RawGasQ = [4000, 5050, 7100, 4300]
Total_Cost = sum([x[0]*x[1] for x in zip(Unit_Cost, RawGasQ)])

#Create list of resell prices for unused raw materials
resell_prices = [36.85, 36.85, 38.95, 38.95]

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Fuel Mix", LpMaximize)

# Create the Variable for percentages of raw gas types in fuel mixes
quantities_vars = LpVariable.dicts("Quantities",Quantities,lowBound=0,upBound=None,cat=LpContinuous)

# The objective function is added to 'prob' first
prob += (lpSum([quantities_vars['R1_{}'.format(i[1])]*Prices[i] for i in Fuels]) +
         lpSum([quantities_vars['R2_{}'.format(i[1])]*Prices[i] for i in Fuels]) +
         lpSum([quantities_vars['R3_{}'.format(i[1])]*Prices[i] for i in Fuels]) +
         lpSum([quantities_vars['R4_{}'.format(i[1])]*Prices[i] for i in Fuels]) +
         (RawGasQ[0] - lpSum([quantities_vars[i] for i in Quantities if 'R1' in i]))* resell_prices[0] -
         (RawGasQ[1] - lpSum([quantities_vars[i] for i in Quantities if 'R2' in i]))* resell_prices[1] -
         (RawGasQ[2] - lpSum([quantities_vars[i] for i in Quantities if 'R3' in i]))* resell_prices[2] -
         (RawGasQ[3] - lpSum([quantities_vars[i] for i in Quantities if 'R4' in i]))* resell_prices[3] -
         Total_Cost), "Total Profit from Fuel Production"

# We can enter the constraints that relate to meeting the demand
for r in Fuels:
    prob += lpSum([quantities_vars[i] for i in Quantities if i[3]==r[1]]) <= MaxQ[r], 'Max_{}'.format(r)
    prob += lpSum([quantities_vars[i] for i in Quantities if i[3]==r[1]]) >= MinQ[r], 'Min_{}'.format(r)

# We can enter the constraints that relate to meeting the octane constraints
for r in Fuels:
    prob += lpSum([quantities_vars['{}_{}'.format(i, r[1])]*OctaneConstraints[r][i] for i in RawGas]) >= 0, 'Octane_{}'.format(r)
    
# We can enter the capacity constraints
for i in RawGas:
    prob += lpSum([quantities_vars['{}_{}'.format(i, r[1])] for r in Fuels]) <= Capacity[i], 'Capacity_{}'.format(i)

# The problem data is written to an .lp file
prob.writeLP("FuelMix.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print("Total Profit of Plan = ", value(prob.objective))

for constraint in prob.constraints:
        print(prob.constraints[constraint].name, prob.constraints[constraint].value())
        print(prob.constraints[constraint].name, prob.constraints[constraint].constant)

Status: Optimal
Quantities_R1_1 = 0.0
Quantities_R1_2 = 0.0
Quantities_R1_3 = 3457.4074
Quantities_R2_1 = 1509.9715
Quantities_R2_2 = 0.0
Quantities_R2_3 = 3540.0285
Quantities_R3_1 = 0.0
Quantities_R3_2 = 0.0
Quantities_R3_3 = 7100.0
Quantities_R4_1 = 3397.4359
Quantities_R4_2 = 0.0
Quantities_R4_3 = 902.5641
Total Profit of Plan =  140431.4814199999
Max_F1 -5092.5926
Max_F1 -10000
Min_F1 4907.4074
Min_F1 0
Max_F2 -999999.0
Max_F2 -999999
Min_F2 0.0
Min_F2 0
Max_F3 -984999.0
Max_F3 -999999
Min_F3 4.547473508864641e-13
Min_F3 -15000
Octane_F1 9.999999929277692e-05
Octane_F1 0
Octane_F2 0.0
Octane_F2 0
Octane_F3 0.00010000000474974513
Octane_F3 0
Capacity_R1 -542.5926
Capacity_R1 -4000
Capacity_R2 0.0
Capacity_R2 -5050
Capacity_R3 0.0
Capacity_R3 -7100
Capacity_R4 0.0
Capacity_R4 -4300


In [43]:
prob

Fuel Mix:
MAXIMIZE
8.299999999999997*Quantities_R1_1 + 6.100000000000001*Quantities_R1_2 + 4.140000000000001*Quantities_R1_3 + 82.0*Quantities_R2_1 + 79.80000000000001*Quantities_R2_2 + 77.84*Quantities_R2_3 + 84.1*Quantities_R3_1 + 81.9*Quantities_R3_2 + 79.94*Quantities_R3_3 + 84.1*Quantities_R4_1 + 81.9*Quantities_R4_2 + 79.94*Quantities_R4_3 + -1198705.0
SUBJECT TO
Max_F1: Quantities_R1_1 + Quantities_R2_1 + Quantities_R3_1 + Quantities_R4_1
 <= 10000

Min_F1: Quantities_R1_1 + Quantities_R2_1 + Quantities_R3_1 + Quantities_R4_1
 >= 0

Max_F2: Quantities_R1_2 + Quantities_R2_2 + Quantities_R3_2 + Quantities_R4_2
 <= 999999

Min_F2: Quantities_R1_2 + Quantities_R2_2 + Quantities_R3_2 + Quantities_R4_2
 >= 0

Max_F3: Quantities_R1_3 + Quantities_R2_3 + Quantities_R3_3 + Quantities_R4_3
 <= 999999

Min_F3: Quantities_R1_3 + Quantities_R2_3 + Quantities_R3_3 + Quantities_R4_3
 >= 15000

Octane_F1: - 27 Quantities_R1_1 - 9 Quantities_R2_1 - 4 Quantities_R3_1
 + 4 Quantities_R4_1 >= 0

O

### Question 5

In [ ]:
#Create a list of all the shifts
Products = ["A","B","C","D","E","F"]

#Create a list of all the machines
Machines = ["M1","M2","M3"]

#Create a Dictionary of Availability
#Same value for each machine!
MachinesAvailability = 336

#Create a list for amount used of each raw material for each product
CapacityUsed = {"A": {"M1": 2, "M2": 0, "M3": 2},
              "B":{"M1": 0, "M2": 2.5, "M3": 1.5}
             }

# Create the 'prob' variable to contain the problem data
prob = LpProblem("2 products 3 machines", LpMaximize)

# Create the Variables
product_vars = {p:LpVariable("Prod_{}".format(p),lowBound=MinQ[p],upBound=140,cat=LpContinuous) for p in Products}

# The objective function is added to 'prob' first
prob += lpSum([Profits[i]*product_vars[i] for i in Products]) - 50000, "Total Revenue of Production Plan"

# We can enter the constraints that relate to limited amount of material
for r in Machines:
    prob += lpSum([product_vars[i]*CapacityUsed[i][r] for i in Products]) <= MachinesAvailability, r

# The problem data is written to an .lp file
prob.writeLP("MachinesProducts.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print("Total Profit of Plan = ", value(prob.objective))

for constraint in prob.constraints:
        print(prob.constraints[constraint].name, prob.constraints[constraint].value())
        print(prob.constraints[constraint].name, prob.constraints[constraint].constant)

### Question 6

In [6]:
def get_waste(p):
    return 20 - (p['a']*5 + p['b']*7 + p['c']*9)

#list all possible patterns that solve the problem
problem = Problem()
problem.addVariable("a", [0,1,2,3,4])
problem.addVariable("b", [0,1,2])
problem.addVariable("c", [0,1,2])
problem.addConstraint(lambda a, b, c: a*5 + b*7 + c*9 <= 20,
                          ("a", "b", "c"))
problem.addConstraint(lambda a, b, c: a*5 + b*7 + c*9 > 15,
                          ("a", "b", "c"))

solutions = problem.getSolutions()
solutions_dict = {"p{}".format(i+1) : solutions[i] for i in range(len(solutions))}

In [7]:
#Create a list of all the patterns
Patterns = list(solutions_dict.keys())

#Create a dictionary of the waste per unit for products
Waste = {Patterns[i] : get_waste(solutions[i]) for i in range(len(solutions))}

#Create a list of all the orders
Orders = ["O1","O2","O3"]

#Create a Dictionary of Minimum quantities per order
MinQuantity = {"O1": 150,
                   "O2": 200,
                   "O3": 300}

#Create a list of how each pattern contributes to satisfying an order
CapacityUsed = {"O1" : {Patterns[i] : solutions[i]['a'] for i in range(len(solutions))},
              "O2" : {Patterns[i] : solutions[i]['b'] for i in range(len(solutions))},
              "O3" : {Patterns[i] : solutions[i]['c'] for i in range(len(solutions))},  
             }

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Cutting stock", LpMinimize)

# Create the Variables
pattern_vars = LpVariable.dicts("Pattern",Patterns,lowBound=0,upBound=None,cat=LpInteger)

# The objective function is added to 'prob' first
prob += lpSum([Waste[i]*pattern_vars[i] for i in Patterns]), "Total Waste of Production Plan"

# We can enter the constraints that relate to limited amount of material
for r in Orders:
    prob += lpSum([pattern_vars[i]*CapacityUsed[r][i] for i in Patterns]) >= MinQuantity[r], r
    
# The problem data is written to an .lp file
prob.writeLP("CuttingStock.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
tot_rolls = 0
for v in prob.variables():
    print(v.name, "=", v.varValue)
    tot_rolls += v.varValue
    
# The optimised objective function value is printed to the screen
print("Total Waste = ", value(prob.objective))

# The number of paper rolls used is printed to the screen
print("Total Number of 20-ft rolls = ", tot_rolls)

for constraint in prob.constraints:
        print(prob.constraints[constraint].name, prob.constraints[constraint].value())
        print(prob.constraints[constraint].name, prob.constraints[constraint].constant)

Status: Optimal
Pattern_p1 = 100.0
Pattern_p2 = 0.0
Pattern_p3 = 0.0
Pattern_p4 = 150.0
Pattern_p5 = 0.0
Pattern_p6 = 13.0
Total Waste =  400.0
Total Number of 20-ft rolls =  263.0
O1 2.0
O1 -150
O2 0.0
O2 -200
O3 0.0
O3 -300


In [8]:
solutions_dict

{'p1': {'a': 1, 'b': 2, 'c': 0},
 'p2': {'a': 0, 'b': 1, 'c': 1},
 'p3': {'a': 2, 'b': 1, 'c': 0},
 'p4': {'a': 0, 'b': 0, 'c': 2},
 'p5': {'a': 2, 'b': 0, 'c': 1},
 'p6': {'a': 4, 'b': 0, 'c': 0}}

In [9]:
prob

Cutting stock:
MINIMIZE
1*Pattern_p1 + 4*Pattern_p2 + 3*Pattern_p3 + 2*Pattern_p4 + 1*Pattern_p5 + 0
SUBJECT TO
O1: Pattern_p1 + 2 Pattern_p3 + 2 Pattern_p5 + 4 Pattern_p6 >= 150

O2: 2 Pattern_p1 + Pattern_p2 + Pattern_p3 >= 200

O3: Pattern_p2 + 2 Pattern_p4 + Pattern_p5 >= 300

VARIABLES
0 <= Pattern_p1 Integer
0 <= Pattern_p2 Integer
0 <= Pattern_p3 Integer
0 <= Pattern_p4 Integer
0 <= Pattern_p5 Integer
0 <= Pattern_p6 Integer